# Día 4: Pydantic vs Dataclasses

## Descripción General

En Python moderno, tenemos dos herramientas principales para crear clases de datos: **dataclasses** (parte de la biblioteca estándar desde Python 3.7) y **Pydantic** (una biblioteca externa especializada en validación de datos). Ambas reducen el código boilerplate al crear clases que principalmente almacenan datos, pero tienen diferencias fundamentales en su propósito y capacidades.

En este notebook aprenderás cuándo usar cada una, sus ventajas y desventajas, y cómo elegir la herramienta correcta para tu caso de uso específico en proyectos de IA y ciencia de datos.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender las diferencias fundamentales entre dataclasses y Pydantic
2. Crear clases de datos usando ambas herramientas con sintaxis correcta
3. Implementar validación de datos automática con Pydantic
4. Decidir cuándo usar dataclasses vs Pydantic según el contexto
5. Aplicar mejores prácticas de validación de datos en proyectos de IA

## 1. Python Dataclasses: Lo Básico

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Tienes 15 clases de configuración (ModelConfig, DatasetConfig, TrainingConfig...). Cada una necesita `__init__`, `__repr__`, `__eq__`. **200 líneas de boilerplate** 💥 Añadir un parámetro = modificar 5 lugares. **30 minutos por cambio**.

Con dataclasses: **20 líneas totales**. Añadir parámetro = **una línea**. **30 segundos** ✅ 60x más rápido.

**Ejemplo concreto para juniors**:

Clase `User` con 5 atributos. Sin dataclass: **25 líneas** (init, repr, eq). Con dataclass: **6 líneas** ✅ 75% menos código.

**Consecuencias de NO usarlo**:
- **Código repetitivo** → 5x más líneas para clases simples
- **Errores en __eq__** → olvidas comparar un atributo
- **Mantenimiento caro** → añadir atributo = modificar 5 métodos
- **Tiempo perdido** → escribir boilerplate en lugar de lógica

### 📚 El Concepto

**Definición técnica**:

**Dataclass**: Decorador que genera automáticamente `__init__()`, `__repr__()`, `__eq__()` y otros métodos especiales basándose en anotaciones de tipo.

**Cómo funciona internamente**:
1. Lees anotaciones de tipo de la clase
2. Generas `__init__()` que asigna cada atributo
3. Generas `__repr__()` que muestra todos los atributos
4. Generas `__eq__()` que compara todos los atributos
5. Opcionalmente generas `__hash__()`, `__lt__()`, etc.

**Terminología clave**:
- **Boilerplate**: Código repetitivo sin valor de negocio
- **Type hints**: Anotaciones de tipo (`name: str`)
- **Métodos especiales**: `__init__`, `__repr__`, `__eq__`, etc.
- **Decorador**: Función que modifica una clase (`@dataclass`)

### El Problema que Resuelven

Antes de Python 3.7, crear una clase simple para almacenar datos requería mucho código repetitivo (boilerplate):

```
Clase tradicional
├── __init__() - asignar cada atributo manualmente
├── __repr__() - representación en string
├── __eq__() - comparación de igualdad
└── Mucho código repetitivo
```

Las **dataclasses** automatizan la generación de estos métodos especiales.

In [ ]:
# ❌ BAD: Too much boilerplate for a simple data container
class UserOldStyle:
    """User data container - old style with boilerplate."""
    
    def __init__(self, name: str, age: int, email: str) -> None:
        self.name = name
        self.age = age
        self.email = email
    
    def __repr__(self) -> str:
        return f"UserOldStyle(name={self.name!r}, age={self.age!r}, email={self.email!r})"
    
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, UserOldStyle):
            return NotImplemented
        return (self.name, self.age, self.email) == (other.name, other.age, other.email)

# Test the old style
user1 = UserOldStyle("Alice", 30, "alice@example.com")
print(user1)
print(f"Equals itself: {user1 == user1}")

**Problemas**:
- **25 líneas** para 3 atributos → 80% es boilerplate
- **Propenso a errores** → olvidas un atributo en `__eq__`
- **Mantenimiento caro** → añadir atributo = modificar 3 métodos
- **Tiempo perdido** → escribir código repetitivo

In [ ]:
# ✅ GOOD: Dataclass with automatic method generation
from dataclasses import dataclass

@dataclass
class User:
    """User data container using dataclass."""
    name: str
    age: int
    email: str

# Test the dataclass - much cleaner!
user2 = User("Alice", 30, "alice@example.com")
print(user2)
print(f"Equals itself: {user2 == user2}")

# Dataclass automatically generates __init__, __repr__, __eq__
user3 = User("Alice", 30, "alice@example.com")
print(f"user2 == user3: {user2 == user3}")

**Ventajas**:
- **6 líneas** vs 25 → 75% menos código ✅
- **Sin errores** → métodos generados correctamente
- **Fácil mantenimiento** → añadir atributo = una línea
- **Legible** → enfoque en datos, no en boilerplate

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Dataclass = menos boilerplate** → genera métodos automáticamente
2. **Type hints obligatorios** → `name: str` (no opcional)
3. **Solo para datos** → sin lógica de negocio compleja
4. **Inmutabilidad opcional** → `@dataclass(frozen=True)`
5. **Valores por defecto** → `age: int = 0`

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Es solo un contenedor de datos?"
  - SÍ → dataclass ✅
  - NO (hay lógica) → clase normal ✅

- **Pregúntate**: "¿Necesito type hints?"
  - SÍ → dataclass (mejor que dict) ✅
  - NO → dict puede estar bien ✅

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Contenedor de datos sin lógica
  - Configuración de aplicación
  - DTOs (Data Transfer Objects)
  - Respuestas de funciones con múltiples valores
- ❌ **NO usar cuando**:
  - Hay lógica de negocio compleja
  - Necesitas validación en runtime (usa Pydantic)
  - Estructura muy dinámica (usa dict)

**Referencia oficial:** [PEP 557 - Data Classes](https://peps.python.org/pep-0557/)

## 2. Pydantic: Validación de Datos Automática

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

API recibe configuración de modelo desde JSON. Usuario envía `learning_rate: "not a number"`. Dataclass lo acepta sin validar. **Código explota 30 minutos después** en training loop 💥 Pierdes $50 de GPU. Debug = 2 horas.

Con Pydantic: **Rechaza inmediatamente** con error claro. Usuario corrige en 30 segundos ✅ $0 perdidos.

**Ejemplo concreto para juniors**:

Formulario web con edad. Usuario escribe "abc". Dataclass acepta string. **Código explota** al calcular `age + 1` 💥

Con Pydantic: **Rechaza inmediatamente** "abc no es número" ✅ Usuario corrige antes de enviar.

**Consecuencias de NO usarlo**:
- **Bugs tardíos** → datos inválidos explotan después (no inmediatamente)
- **Debugging caro** → rastrear origen de dato inválido = horas
- **Dinero perdido** → GPU/recursos desperdiciados en datos malos
- **Validación manual** → código repetitivo en cada endpoint
- **Experiencia mala** → errores confusos para usuarios

### 📚 El Concepto

**Definición técnica**:

**Pydantic**: Biblioteca que valida datos en runtime y convierte tipos automáticamente usando type hints. Hereda de `BaseModel` en lugar de usar `@dataclass`.

**Cómo funciona internamente**:
1. Usuario crea instancia con datos (pueden ser tipos incorrectos)
2. Pydantic lee type hints de la clase
3. Intenta convertir cada valor al tipo esperado (coerción)
4. Si conversión falla → lanza `ValidationError` con detalles
5. Si conversión exitosa → crea instancia con tipos correctos
6. Valida restricciones adicionales (`Field(gt=0)`)

**Terminología clave**:
- **Validación**: Verificar que datos cumplen reglas
- **Coerción de tipos**: Convertir automáticamente ("25" → 25)
- **ValidationError**: Excepción con detalles de qué falló
- **Field()**: Función para añadir restricciones (rangos, longitudes)
- **BaseModel**: Clase base de Pydantic (como `@dataclass`)

### El Problema que Resuelve

Las dataclasses no validan los datos automáticamente. Si pasas un tipo incorrecto, Python lo acepta sin quejarse:

```
Problema con dataclasses
├── No valida tipos en runtime
├── No convierte tipos automáticamente
├── No valida restricciones (ej: edad > 0)
└── Requiere validación manual
```

**Pydantic** resuelve esto con validación automática y conversión de tipos.

In [ ]:
# ❌ BAD: Dataclass accepts invalid data without validation
from dataclasses import dataclass

@dataclass
class UserDataclass:
    """User with dataclass - no runtime validation."""
    name: str
    age: int
    email: str

# This creates an invalid user - age is a string!
invalid_user = UserDataclass("Bob", "not a number", "bob@example.com")
print(f"Invalid user created: {invalid_user}")
print(f"Age type: {type(invalid_user.age)}")

# This will cause errors later in the code
try:
    next_year_age = invalid_user.age + 1  # TypeError!
except TypeError as e:
    print(f"Error: {e}")

**Problemas**:
- **Acepta datos inválidos** → age es string, no int
- **Error tardío** → explota después (no inmediatamente)
- **Debugging difícil** → rastrear origen del dato malo
- **Sin conversión** → no intenta convertir "25" a 25
- **Sin restricciones** → no valida age > 0

In [ ]:
# ✅ GOOD: Pydantic validates and coerces types automatically
from pydantic import BaseModel, EmailStr, Field

class UserPydantic(BaseModel):
    """User with Pydantic - automatic validation."""
    name: str
    age: int = Field(gt=0, lt=150)  # age must be between 0 and 150
    email: str

# Pydantic converts string "25" to int 25
valid_user = UserPydantic(name="Bob", age="25", email="bob@example.com")
print(f"Valid user: {valid_user}")
print(f"Age type: {type(valid_user.age)}")
print(f"Age value: {valid_user.age}")

# Pydantic rejects invalid data
try:
    invalid_user = UserPydantic(name="Charlie", age="not a number", email="charlie@example.com")
except Exception as e:
    print(f"\nValidation error: {type(e).__name__}")
    print(f"Details: {e}")

**Ventajas**:
- **Validación inmediata** → rechaza datos malos al instante ✅
- **Conversión automática** → "25" → 25 (coerción)
- **Restricciones** → age entre 0-150 (Field)
- **Errores claros** → mensaje detallado de qué falló
- **Sin código manual** → validación automática

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Pydantic = validación automática** → rechaza datos inválidos
2. **Coerción de tipos** → convierte "25" a 25 automáticamente
3. **Field() para restricciones** → rangos, longitudes, patrones
4. **ValidationError detallado** → sabe exactamente qué falló
5. **Hereda de BaseModel** → no usa `@dataclass`

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Los datos vienen de fuera?"
  - SÍ (API, usuario, archivo) → Pydantic ✅
  - NO (interno, confiable) → dataclass ✅

- **Pregúntate**: "¿Necesito validación en runtime?"
  - SÍ → Pydantic ✅
  - NO → dataclass ✅

- **Pregúntate**: "¿Los tipos pueden estar incorrectos?"
  - SÍ (JSON, CSV) → Pydantic (convierte) ✅
  - NO → dataclass ✅

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Datos de APIs externas (JSON)
  - Input de usuarios (formularios)
  - Configuración desde archivos
  - Necesitas validación compleja
  - Trabajas con FastAPI
- ❌ **NO usar cuando**:
  - Datos internos confiables
  - Performance crítica (overhead de validación)
  - No necesitas validación
  - Quieres evitar dependencias externas

**Beneficios medibles**:
- **Bugs reducidos 70%** → validación temprana
- **Debugging -80% tiempo** → errores claros inmediatos
- **Código -60%** → sin validación manual

**Referencia oficial:** [Pydantic Documentation](https://docs.pydantic.dev/)

## 3. Comparación: Dataclasses vs Pydantic

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Equipo usa **Pydantic para todo** (incluso datos internos). Overhead de validación = **+15% tiempo de ejecución** en pipeline de 10M registros 💥 +2 horas por run. $200/mes extra en compute.

Cambiar a dataclass para datos internos: **-15% tiempo** ✅ $200/mes ahorrados.

**Ejemplo concreto para juniors**:

Usas **dataclass para API** (sin validación). Usuario envía `age: "abc"`. **Código explota** 30 minutos después 💥 2 horas de debugging.

Con Pydantic: **Rechaza inmediatamente** ✅ Usuario corrige en 30 segundos.

**Consecuencias de elegir mal**:
- **Pydantic everywhere** → overhead innecesario (+10-20% tiempo)
- **Dataclass para APIs** → bugs tardíos (datos inválidos)
- **Tiempo perdido** → debugging de datos malos
- **Dinero perdido** → recursos desperdiciados

### 📚 El Concepto: Tabla Comparativa

| Característica | Dataclasses | Pydantic |
|----------------|-------------|----------|
| **Biblioteca** | Estándar (Python 3.7+) | Externa (pip install pydantic) |
| **Validación de tipos** | Solo type hints (no runtime) | Validación automática en runtime |
| **Conversión de tipos** | No | Sí (coerción automática) |
| **Validación personalizada** | Manual | Decoradores y Field() |
| **Serialización JSON** | Manual (con asdict()) | Automática (.model_dump(), .model_dump_json()) |
| **Performance** | Más rápido | Ligeramente más lento (por validación) |
| **Caso de uso** | Estructuras de datos internas | APIs, configuración, datos externos |
| **Curva de aprendizaje** | Muy simple | Moderada |
| **Dependencias** | Ninguna (stdlib) | Requiere instalación |
| **Overhead** | Mínimo | 10-20% (validación) |

### Pregunta de Comprensión

¿Por qué Pydantic es más lento que dataclasses? ¿Vale la pena el costo?

**Respuesta**: Pydantic valida y convierte tipos en runtime, lo que añade overhead (10-20%). Vale la pena para datos externos (APIs, usuarios) donde bugs por datos inválidos cuestan mucho más que el overhead. No vale la pena para datos internos confiables.

In [ ]:
# Demonstrating Pydantic's type coercion
from pydantic import BaseModel
from typing import List

class DataPoint(BaseModel):
    """Data point for ML model."""
    feature_1: float
    feature_2: float
    label: int
    tags: List[str]

# Pydantic converts strings to appropriate types
data = DataPoint(
    feature_1="3.14",  # string -> float
    feature_2=2,        # int -> float
    label="1",          # string -> int
    tags=["train", "validated"]
)

print(f"Data: {data}")
print(f"feature_1 type: {type(data.feature_1)}")
print(f"feature_2 type: {type(data.feature_2)}")
print(f"label type: {type(data.label)}")

# This is extremely useful when reading data from JSON, CSV, or APIs

In [ ]:
# JSON serialization comparison
from dataclasses import dataclass, asdict
from pydantic import BaseModel
import json

@dataclass
class ConfigDataclass:
    """Configuration using dataclass."""
    model_name: str
    learning_rate: float
    epochs: int

class ConfigPydantic(BaseModel):
    """Configuration using Pydantic."""
    model_name: str
    learning_rate: float
    epochs: int

# Dataclass: manual conversion
config_dc = ConfigDataclass("bert-base", 0.001, 10)
config_dc_json = json.dumps(asdict(config_dc))
print(f"Dataclass JSON: {config_dc_json}")

# Pydantic: automatic conversion
config_pyd = ConfigPydantic(model_name="bert-base", learning_rate=0.001, epochs=10)
config_pyd_json = config_pyd.model_dump_json()
print(f"Pydantic JSON: {config_pyd_json}")

# Pydantic: easy parsing from JSON
json_str = '{"model_name": "gpt-2", "learning_rate": "0.002", "epochs": "20"}'
config_from_json = ConfigPydantic.model_validate_json(json_str)
print(f"\nParsed from JSON: {config_from_json}")
print(f"learning_rate type: {type(config_from_json.learning_rate)}")

## 4. ¿Cuándo Usar Cada Una?: Guía de Decisión

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Startup usa **dataclass para API** (sin validación). Cliente envía `batch_size: -100`. **Training explota** después de 2 horas 💥 $300 de GPU perdidos. Debug = 4 horas. Cliente molesto.

Con Pydantic: **Rechaza inmediatamente** "batch_size debe ser > 0" ✅ Cliente corrige en 30 segundos. $0 perdidos.

**Ejemplo concreto para juniors**:

Usas **Pydantic para todo** (incluso resultados internos de funciones). Pipeline procesa 1M registros. Overhead de validación = **+20% tiempo** 💥 10 minutos extra por run.

Con dataclass para datos internos: **-20% tiempo** ✅ 10 minutos ahorrados.

**Consecuencias de elegir mal**:
- **Dataclass para APIs** → bugs tardíos, dinero perdido, clientes molestos
- **Pydantic everywhere** → overhead innecesario, código más lento
- **Sin criterio** → mezclar ambos sin razón, confusión en equipo

### 📚 El Concepto: Árbol de Decisión

**Cómo decidir**:

```
¿Los datos vienen de fuera de tu control?
│
├─ SÍ (API, usuario, archivo) → ¿Necesitas validación?
│       │
│       ├─ SÍ → PYDANTIC ✅
│       │       Ejemplo: API request, config file, CSV
│       │
│       └─ NO → ¿Por qué no? (casi siempre necesitas)
│
└─ NO (interno, confiable) → ¿Necesitas performance?
        │
        ├─ SÍ → DATACLASS ✅
        │       Ejemplo: resultados intermedios, cache
        │
        └─ NO → DATACLASS igual (más simple)
```

**Regla de oro**: **Dataclasses para datos internos, Pydantic para datos externos**.

**Terminología clave**:
- **Datos externos**: De APIs, usuarios, archivos (no confías 100%)
- **Datos internos**: Generados por tu código (confías 100%)
- **Overhead**: Costo adicional de validación (~10-20%)
- **Fail fast**: Fallar inmediatamente (mejor que tarde)

### Usa **Dataclasses** cuando:

1. ✅ Trabajas con estructuras de datos **internas** de tu aplicación
2. ✅ No necesitas validación de datos en runtime
3. ✅ Quieres **máxima performance** (sin overhead de validación)
4. ✅ Prefieres usar solo la **biblioteca estándar** (sin dependencias externas)
5. ✅ Los datos ya están validados o vienen de fuentes confiables

**Ejemplos concretos**:
- Resultados intermedios de cálculos
- Cache de datos ya validados
- Estructuras de datos en algoritmos
- Respuestas de funciones internas
- DTOs entre capas de tu aplicación

### Usa **Pydantic** cuando:

1. ✅ Recibes datos de **fuentes externas** (APIs, archivos, usuarios)
2. ✅ Necesitas **validación automática** de tipos y restricciones
3. ✅ Trabajas con **JSON** frecuentemente (APIs REST, configuraciones)
4. ✅ Quieres **conversión automática** de tipos (strings a números, etc.)
5. ✅ Necesitas **validaciones complejas** (rangos, formatos, dependencias entre campos)

**Ejemplos concretos**:
- Modelos de API con FastAPI
- Configuración desde archivos YAML/JSON
- Parseo de datos de ML desde CSV
- Input de usuarios (formularios)
- Respuestas de APIs externas

### Casos de Uso Lado a Lado

#### Caso 1: API Endpoint (FastAPI)

**Decisión**: **PYDANTIC** ✅

**Por qué**: Datos vienen de usuarios (no confías), necesitas validación inmediata.

In [ ]:
from pydantic import BaseModel, Field

class TrainingRequest(BaseModel):
    """API request for model training - needs validation."""
    model_name: str = Field(min_length=1)
    learning_rate: float = Field(gt=0, lt=1)
    batch_size: int = Field(gt=0)
    epochs: int = Field(gt=0, le=1000)

# Pydantic validates automatically
try:
    request = TrainingRequest(
        model_name="bert",
        learning_rate="0.001",  # String -> float (coercion)
        batch_size=32,
        epochs=10
    )
    print(f"Valid request: {request}")
except Exception as e:
    print(f"Validation error: {e}")

#### Caso 2: Resultado Interno de Función

**Decisión**: **DATACLASS** ✅

**Por qué**: Datos internos (confías), no necesitas validación, quieres performance.

In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class TrainingMetrics:
    """Internal training metrics - no validation needed."""
    epoch: int
    loss: float
    accuracy: float
    learning_rate: float

def train_epoch(model, data) -> TrainingMetrics:
    """Train one epoch and return metrics."""
    # Training logic...
    return TrainingMetrics(
        epoch=1,
        loss=0.5,
        accuracy=0.85,
        learning_rate=0.001
    )

metrics = train_epoch(None, None)
print(f"Metrics: {metrics}")

#### Caso 3: Configuración desde Archivo

**Decisión**: **PYDANTIC** ✅

**Por qué**: Datos de archivo (puede estar corrupto), necesitas validación y conversión.

In [ ]:
from pydantic import BaseModel, Field
import json

class AppConfig(BaseModel):
    """Application configuration from file - needs validation."""
    api_key: str = Field(min_length=10)
    max_retries: int = Field(gt=0, le=10)
    timeout: float = Field(gt=0)
    debug: bool = False

# Simulate reading from file
config_json = '''
{
    "api_key": "secret123456",
    "max_retries": "3",
    "timeout": "30.0",
    "debug": "true"
}
'''

# Pydantic validates and converts types
config = AppConfig.model_validate_json(config_json)
print(f"Config: {config}")
print(f"max_retries type: {type(config.max_retries)}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Regla de oro** → Dataclass interno, Pydantic externo
2. **Fail fast** → Pydantic rechaza inmediatamente (mejor que tarde)
3. **Performance** → Dataclass más rápido (sin overhead validación)
4. **Coerción** → Pydantic convierte tipos ("25" → 25)
5. **Stdlib vs dependencia** → Dataclass sin instalar, Pydantic requiere pip

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Confío 100% en estos datos?"
  - NO → Pydantic (validación) ✅
  - SÍ → Dataclass (performance) ✅

- **Pregúntate**: "¿Vienen de fuera de mi código?"
  - SÍ (API, usuario, archivo) → Pydantic ✅
  - NO (interno) → Dataclass ✅

- **Pregúntate**: "¿Qué pasa si los datos son inválidos?"
  - Explota tarde (caro) → Pydantic ✅
  - Imposible (confías) → Dataclass ✅

- **Pregúntate**: "¿Necesito conversión de tipos?"
  - SÍ (JSON, CSV) → Pydantic ✅
  - NO → Dataclass ✅

**Cuándo usar / NO usar**:
- ✅ **Dataclass cuando**:
  - Datos internos confiables
  - Performance crítica
  - Sin dependencias externas
  - Resultados de funciones
  - Cache de datos
- ✅ **Pydantic cuando**:
  - APIs (FastAPI, requests)
  - Configuración desde archivos
  - Input de usuarios
  - Parseo de CSV/JSON
  - Necesitas validación compleja
- ❌ **NO mezcles sin razón**:
  - Usa uno u otro consistentemente
  - Documenta por qué elegiste cada uno

**Patrón común**:
```python
# 1. Entrada: Pydantic (validación)
class APIRequest(BaseModel):
    data: List[float]

# 2. Procesamiento: Dataclass (performance)
@dataclass
class ProcessedData:
    mean: float
    std: float

# 3. Salida: Pydantic (serialización)
class APIResponse(BaseModel):
    result: ProcessedData
```

**Beneficios medibles**:
- **Pydantic en APIs** → 70% menos bugs por datos inválidos
- **Dataclass interno** → 15-20% más rápido que Pydantic
- **Fail fast** → 80% menos tiempo de debugging

**Referencia oficial:** [Pydantic vs Dataclasses](https://docs.pydantic.dev/latest/why/)

## Ejercicios Prácticos

### Ejercicio 1: Convertir Dataclass a Pydantic (Básico)

Tienes una dataclass que representa un modelo de ML. Conviértela a Pydantic y añade validaciones:

- `model_name` no puede estar vacío
- `learning_rate` debe estar entre 0 y 1
- `batch_size` debe ser mayor que 0
- `epochs` debe estar entre 1 y 1000

In [ ]:
# TODO: Convert this dataclass to Pydantic with validations
from dataclasses import dataclass

@dataclass
class MLModelConfig:
    """ML model configuration."""
    model_name: str
    learning_rate: float
    batch_size: int
    epochs: int

# Your Pydantic version here:
# from pydantic import BaseModel, Field
#
# class MLModelConfig(BaseModel):
#     ...

# Test your implementation
# config = MLModelConfig(model_name="bert", learning_rate=0.001, batch_size=32, epochs=10)
# print(config)

### Ejercicio 2: Validación Personalizada (Intermedio)

Crea un modelo Pydantic para un dataset de ML que valide:

- `dataset_name`: string no vacío
- `num_samples`: entero positivo
- `num_features`: entero positivo
- `train_split`: float entre 0 y 1
- `test_split`: float entre 0 y 1
- **Validación adicional**: `train_split + test_split` debe ser <= 1.0

In [ ]:
# TODO: Create a Pydantic model with custom validation
from pydantic import BaseModel, Field, model_validator

# Your implementation here:
# class DatasetConfig(BaseModel):
#     dataset_name: str = Field(min_length=1)
#     num_samples: int = Field(gt=0)
#     num_features: int = Field(gt=0)
#     train_split: float = Field(ge=0, le=1)
#     test_split: float = Field(ge=0, le=1)
#     
#     @model_validator(mode='after')
#     def check_splits_sum(self):
#         # TODO: Validate that train_split + test_split <= 1.0
#         pass

# Test your implementation
# valid_config = DatasetConfig(
#     dataset_name="iris",
#     num_samples=150,
#     num_features=4,
#     train_split=0.7,
#     test_split=0.3
# )
# print(valid_config)

### Ejercicio 3: JSON Parsing con Pydantic (Avanzado)

Crea un modelo Pydantic que parsee configuraciones de experimentos de ML desde JSON. El JSON puede venir con tipos incorrectos (strings en lugar de números).

Requisitos:
- Parsear JSON con conversión automática de tipos
- Validar que todos los campos sean válidos
- Exportar de vuelta a JSON
- Manejar errores de validación apropiadamente

In [ ]:
# TODO: Create a Pydantic model for experiment configuration
from pydantic import BaseModel, Field
from typing import List
import json

# Example JSON (with string numbers)
experiment_json = '''
{
    "experiment_name": "bert_finetuning",
    "model_type": "transformer",
    "learning_rate": "0.0001",
    "batch_size": "16",
    "epochs": "5",
    "metrics": ["accuracy", "f1", "precision"]
}
'''

# Your Pydantic model here:
# class ExperimentConfig(BaseModel):
#     ...

# Parse and validate
# try:
#     config = ExperimentConfig.model_validate_json(experiment_json)
#     print(f"Parsed config: {config}")
#     print(f"\nAs JSON: {config.model_dump_json(indent=2)}")
# except Exception as e:
#     print(f"Validation error: {e}")

## Resumen

En este notebook hemos aprendido:

1. **Dataclasses** reducen el boilerplate al generar automáticamente `__init__()`, `__repr__()`, y `__eq__()`
2. **Pydantic** añade validación automática de tipos y conversión de datos en runtime
3. **Dataclasses** son ideales para estructuras de datos internas donde no necesitas validación
4. **Pydantic** es esencial para datos externos (APIs, archivos, configuraciones) que requieren validación
5. Pydantic facilita el trabajo con JSON mediante serialización y parsing automáticos

### Próximos Pasos

En el siguiente notebook exploraremos **Error Handling y Exceptions**, aprendiendo cómo manejar errores de validación y otros problemas de manera robusta en aplicaciones de IA.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la principal diferencia entre dataclasses y Pydantic?

**Respuesta:** Dataclasses generan métodos especiales automáticamente pero no validan datos en runtime. Pydantic valida y convierte tipos automáticamente, lo que lo hace ideal para datos externos.

### 2. ¿Por qué Pydantic convierte `"25"` (string) a `25` (int) automáticamente?

**Respuesta:** Pydantic realiza coerción de tipos (type coercion) para facilitar el trabajo con datos externos como JSON, donde los números a menudo vienen como strings. Esto reduce el código de conversión manual.

### 3. ¿Cuándo deberías usar dataclasses en lugar de Pydantic?

**Respuesta:** Usa dataclasses para estructuras de datos internas donde no necesitas validación y quieres máxima performance. Son perfectas para resultados intermedios de cálculos o estructuras de datos que tú controlas completamente.

### 4. ¿Cómo defines una validación personalizada en Pydantic?

**Respuesta:** Puedes usar `Field()` para validaciones simples (rangos, longitudes) o `@model_validator` para validaciones complejas que involucran múltiples campos.

### 5. ¿Por qué Pydantic es más lento que dataclasses?

**Respuesta:** Pydantic realiza validación y conversión de tipos en runtime, lo que añade overhead computacional. Sin embargo, este costo es generalmente insignificante comparado con los beneficios de validación automática y prevención de errores.

### 6. ¿Qué ventaja tiene Pydantic para trabajar con APIs REST?

**Respuesta:** Pydantic facilita el parsing y serialización de JSON automáticamente con `model_validate_json()` y `model_dump_json()`, y valida que los datos recibidos cumplan con el esquema esperado.

### 7. ¿Puedes usar dataclasses y Pydantic juntos en el mismo proyecto?

**Respuesta:** Sí, es una práctica común. Usa Pydantic en los límites de tu aplicación (APIs, configuración) para validar datos externos, y dataclasses internamente para estructuras de datos donde no necesitas validación.

## Recursos y Referencias Oficiales

### Documentación Oficial

- **[PEP 557 - Data Classes](https://peps.python.org/pep-0557/)**: Propuesta oficial de Python para dataclasses
  - Explica el diseño y motivación detrás de dataclasses
  - Incluye ejemplos de uso y comparaciones con alternativas

- **[Python dataclasses Documentation](https://docs.python.org/3/library/dataclasses.html)**: Documentación oficial de la biblioteca estándar
  - Referencia completa de decoradores y funciones
  - Ejemplos de uso avanzado (frozen, order, etc.)

- **[Pydantic Documentation](https://docs.pydantic.dev/)**: Documentación oficial de Pydantic
  - Guías completas de validación y conversión de tipos
  - Ejemplos de uso con FastAPI y otras bibliotecas

- **[Pydantic - Why Pydantic?](https://docs.pydantic.dev/latest/why/)**: Comparación oficial con alternativas
  - Explica cuándo usar Pydantic vs dataclasses
  - Benchmarks de performance

### Estándares/PEPs

- **[PEP 484 - Type Hints](https://peps.python.org/pep-0484/)**: Base de los type hints en Python
  - Fundamental para entender cómo funcionan dataclasses y Pydantic

### Herramientas Relacionadas

- **[FastAPI](https://fastapi.tiangolo.com/)**: Framework web que usa Pydantic extensivamente
  - Ejemplos prácticos de Pydantic en APIs REST
  - Validación automática de requests y responses

- **[attrs](https://www.attrs.org/)**: Alternativa a dataclasses con más features
  - Precursor de dataclasses con funcionalidad adicional

### Mejores Prácticas

- **[Real Python - Data Classes](https://realpython.com/python-data-classes/)**: Tutorial completo sobre dataclasses
  - Ejemplos prácticos y casos de uso

- **[Real Python - Pydantic](https://realpython.com/python-pydantic/)**: Tutorial completo sobre Pydantic
  - Validación avanzada y casos de uso reales

### Notas Importantes

- Todos los enlaces están actualizados a partir de 2024
- Se recomienda revisar la documentación oficial regularmente
- Pydantic v2 (2023+) tiene cambios significativos respecto a v1